In [ ]:
import DeepFMKit.core as dfm
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import multiprocessing
import os
import pickle

# Import the new worker function from your workers.py file
from DeepFMKit.workers import calculate_single_distortion_bias

def generate_nonlinearity_bias_data(
    m_true=20.0,
    m_witness_fixed=1.0,
    distortion_range=np.linspace(0, 0.2, 11),
    n_phase_trials=100,
    wdfmi_fitter_name='fit_wdfmi_orthogonal_demodulation',
    n_cores=None
):
    """
    Orchestrates the Monte Carlo analysis of bias vs. modulation non-linearity.
    """
    if n_cores is None:
        n_cores = os.cpu_count()

    print("="*60)
    print("Analyzing Bias from Modulation Non-Linearity")
    print(f"W-DFMI Fitter: {wdfmi_fitter_name}")
    print(f"Distortion Levels: {len(distortion_range)}")
    print(f"MC Trials per Level: {n_phase_trials}")
    print("="*60)
    
    jobs = []
    for dist_amp in distortion_range:
        for _ in range(n_phase_trials):
            jobs.append({
                'm_main': m_true,
                'm_witness': m_witness_fixed,
                'distortion_amp': dist_amp,
                'distortion_phase': np.random.uniform(0, 2 * np.pi),
                'wdfmi_fitter_name': wdfmi_fitter_name
            })

    # Run all jobs in parallel
    results_list = []
    if __name__ == "__main__":
        with multiprocessing.Pool(processes=n_cores) as pool:
            results_iterator = pool.imap(calculate_single_distortion_bias, jobs)
            for result in tqdm(results_iterator, total=len(jobs), desc="Running Distortion Trials"):
                results_list.append(result)

    # --- Process and Structure the Results ---
    wdfmi_biases_by_level = {eps: [] for eps in distortion_range}
    dfmi_biases_by_level = {eps: [] for eps in distortion_range}

    for i, job in enumerate(jobs):
        dist_level = job['distortion_amp']
        wdfmi_bias, dfmi_bias = results_list[i]
        wdfmi_biases_by_level[dist_level].append(wdfmi_bias)
        dfmi_biases_by_level[dist_level].append(dfmi_bias)

    # Calculate final statistics
    wdfmi_mean = np.array([np.nanmean(wdfmi_biases_by_level[eps]) for eps in distortion_range])
    wdfmi_std = np.array([np.nanstd(wdfmi_biases_by_level[eps]) for eps in distortion_range])
    dfmi_mean = np.array([np.nanmean(dfmi_biases_by_level[eps]) for eps in distortion_range])
    dfmi_std = np.array([np.nanstd(dfmi_biases_by_level[eps]) for eps in distortion_range])
    
    final_data = {
        "m_true": m_true,
        "distortion_range": distortion_range,
        "wdfmi_mean_bias": wdfmi_mean,
        "wdfmi_std_bias": wdfmi_std,
        "dfmi_mean_bias": dfmi_mean,
        "dfmi_std_bias": dfmi_std,
        "fitter_name": wdfmi_fitter_name
    }
    
    return final_data

In [ ]:
# --- Main execution block ---
if __name__ == "__main__":
    results_filename = 'nonlinearity_analysis.pkl'
    
    # --- Run Simulation ---
    nonlinearity_results = generate_nonlinearity_bias_data(
        # wdfmi_fitter_name='fit_wdfmi',
        wdfmi_fitter_name='fit_wdfmi_sequential',
        # wdfmi_fitter_name='fit_wdfmi_orthogonal_demodulation',
        n_phase_trials=50 # Lower for faster testing
    )
    
    # --- Save and Plot ---
    with open(results_filename, 'wb') as f:
        pickle.dump(nonlinearity_results, f)

In [ ]:
def plot_nonlinearity_bias(results, wavelength_nm=1064):
    """
    Plots the final comparison of W-DFMI vs DFMI bias from non-linearity.

    This version is updated to plot the "Coarse Phase Error" on the y-axis,
    directly visualizing the Ambiguity Resolution Condition from the manuscript.
    The error is calculated from the bias on 'm' using the relation:
    delta_Phi_coarse = -delta_m * (f0 / df).
    """
    dist_range = results['distortion_range']
    m_true = results['m_true']
    
    # --- Y-Axis Conversion ---
    # According to the theory, the key quantity for ambiguity resolution is the
    # coarse phase error, delta_Phi_coarse. I will convert the bias on m (delta_m)
    # to this quantity.
    # From Eq. (34): Phi_coarse = -m * (f0 / df)
    # Therefore, delta_Phi_coarse = -delta_m * (f0 / df)

    # First, I need the laser carrier frequency, f0.
    f0 = dfm.sc.c / (wavelength_nm * 1e-9)
    
    # Next, I need the modulation amplitude, df, for the given m_true.
    opd_main = dfm.InterferometerConfig().meas_arml - dfm.InterferometerConfig().ref_arml
    df_true = (m_true * dfm.sc.c) / (2 * np.pi * opd_main)
    
    # Now, I can define the conversion factor.
    m_to_phase_error_factor = -f0 / df_true
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # --- Plotting W-DFMI Results (in terms of phase error) ---
    wdfmi_phase_error_mean = results['wdfmi_mean_bias'] * m_to_phase_error_factor
    wdfmi_phase_error_std = results['wdfmi_std_bias'] * m_to_phase_error_factor
    
    ax.plot(dist_range * 100, wdfmi_phase_error_mean, 'o-', color='tab:blue', lw=2.5, label='W-DFMI Mean Error')
    ax.fill_between(dist_range * 100,
                    wdfmi_phase_error_mean - wdfmi_phase_error_std,
                    wdfmi_phase_error_mean + wdfmi_phase_error_std,
                    color='tab:blue', alpha=0.2, label='W-DFMI ±1σ')
                    
    # --- Plotting Conventional DFMI Results (in terms of phase error) ---
    dfmi_phase_error_mean = results['dfmi_mean_bias'] * m_to_phase_error_factor
    dfmi_phase_error_std = results['dfmi_std_bias'] * m_to_phase_error_factor
    
    ax.plot(dist_range * 100, dfmi_phase_error_mean, 's-', color='tab:red', lw=2.5, label='Conventional DFMI Mean Error')
    ax.fill_between(dist_range * 100,
                    dfmi_phase_error_mean - dfmi_phase_error_std,
                    dfmi_phase_error_mean + dfmi_phase_error_std,
                    color='tab:red', alpha=0.2, label='Conventional DFMI ±1σ')

    # --- Ambiguity Limit ---
    # The plot now directly shows the Ambiguity Resolution Condition.
    # A "fringe slip" occurs if the absolute coarse phase error exceeds pi.
    ambiguity_limit_rad = np.pi
    ax.axhline(ambiguity_limit_rad, color='k', linestyle='--', linewidth=2, label=r'Ambiguity Limit ($\pm\pi$)')
    ax.axhline(-ambiguity_limit_rad, color='k', linestyle='--', linewidth=2)
    
    # --- Labels and Title ---
    ax.set_xlabel('2nd Harmonic Distortion Amplitude (%)', fontsize=14)
    ax.set_ylabel(r'Coarse Phase Error, $\delta\Phi_{\rm coarse}$ (rad)', fontsize=14)
    ax.set_title(f'Systematic Error from Modulation Non-Linearity ($m = {m_true}$)', fontsize=16)
    ax.grid(True, which='both', linestyle=':')
    # ax.set_ylim([-2*np.pi,2*np.pi])
    ax.legend(fontsize=12)
    plt.tight_layout()
    plt.show()

In [ ]:
with open(results_filename, 'rb') as f:
    loaded_results = pickle.load(f)
plot_nonlinearity_bias(loaded_results)

In [ ]:
def plot_wdfmi_nonlinearity_bias(results, wavelength_nm=1064):
    dist_range = results['distortion_range']
    m_true = results['m_true']
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # --- Plotting W-DFMI Results (in terms of phase error) ---
    wdfmi_phase_error_mean = results['wdfmi_mean_bias'] 
    wdfmi_phase_error_std = results['wdfmi_std_bias']
    
    ax.plot(dist_range * 100, wdfmi_phase_error_mean, 'o-', color='tab:blue', lw=2.5, label='W-DFMI Mean Error')
    ax.fill_between(dist_range * 100,
                    wdfmi_phase_error_mean - wdfmi_phase_error_std,
                    wdfmi_phase_error_mean + wdfmi_phase_error_std,
                    color='tab:blue', alpha=0.2, label='W-DFMI ±1σ')
                    

    # --- Ambiguity Limit ---
    # The plot now directly shows the Ambiguity Resolution Condition.
    # A "fringe slip" occurs if the absolute coarse phase error exceeds pi.
    ambiguity_limit_rad = np.pi
    ax.axhline(ambiguity_limit_rad, color='k', linestyle='--', linewidth=2, label=r'Ambiguity Limit ($\pm\pi$)')
    ax.axhline(-ambiguity_limit_rad, color='k', linestyle='--', linewidth=2)
    
    # --- Labels and Title ---
    ax.set_xlabel('2nd Harmonic Distortion Amplitude (%)', fontsize=14)
    ax.set_ylabel(r'Coarse Phase Error, $\delta\Phi_{\rm coarse}$ (rad)', fontsize=14)
    ax.set_title(f'Systematic Error from Modulation Non-Linearity ($m = {m_true}$)', fontsize=16)
    ax.grid(True, which='both', linestyle=':')
    # ax.set_ylim([-2*np.pi,2*np.pi])
    ax.legend(fontsize=12)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_wdfmi_nonlinearity_bias(loaded_results)